**Code to pull data from PT and save in Workday upload format**

Installing required libraries

In [ ]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from bs4 import BeautifulSoup
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from dateutil.parser import parse
import pandas as pd
import time
import shutil
import os
from google.colab import userdata
import pathlib
import numpy as np
import re
import sys
from IPython.display import clear_output

Pre-Run Checklist: \\
Upload Mapping Master and Daily PT Data 2024 (linked template) \\
Select which links to groups to run \\
Define start and end date of run \\
Check which locations sunday data is being included for, and which locations no
data should be uploaded for \\

In [ ]:

  # @title Enter PT User Details
USR = input("Enter PT Username: ")
PW = input("Enter PT Password: ")
time.sleep(5)
clear_output()

Links to groups/practices to scrape

In [ ]:
# # means line is hidden (won't run)
links = [
        # ['Mark Chatham', 'https://app.performancetracker.co.uk/switch-group/52'],
        #  ['Penny Goddard', 'https://app.performancetracker.co.uk/switch-group/27'],
        #  ['Sarah Mendham', 'https://app.performancetracker.co.uk/switch-group/88'],
        #  ['Damien Wallwork', 'https://app.performancetracker.co.uk/switch-group/82'],
        #  ['David Thompson', 'https://app.performancetracker.co.uk/switch-group/81'],
        #  ['Kim Whittet', 'https://app.performancetracker.co.uk/switch-group/65'],
        #  ['Hannah Roberts', 'https://app.performancetracker.co.uk/switch-group/97'],
        #  ['Dean Aldis', 'https://app.performancetracker.co.uk/switch-group/89'],
        #  ['Fiona Halliday', 'https://app.performancetracker.co.uk/switch-group/95'],
        #  ['Lynn Blacklaws', 'https://app.performancetracker.co.uk/groups/155'],
        #  ['Paul Foxall', 'https://app.performancetracker.co.uk/switch-group/112'],
        #  ['Angela Campbell', 'https://app.performancetracker.co.uk/switch-group/24'],
        #  ['Chris Frost', 'https://app.performancetracker.co.uk/switch-group/140'],
        #  ['Paul Mothershaw', 'https://app.performancetracker.co.uk/switch-group/26'],
        #  ['Paul Forsythe', 'https://app.performancetracker.co.uk/switch-group/83'],
        #  ['Hayley Gardner-Clark', 'https://app.performancetracker.co.uk/switch-group/29'],
        #  ['Janine Andrews', 'https://app.performancetracker.co.uk/switch-group/122'],
        #  ['Daniel Martin', 'https://app.performancetracker.co.uk/switch-group/121'],
        #  ['Sally Whitworth', 'https://app.performancetracker.co.uk/switch-group/115'],
        #  ['Stephen Potter', 'https://app.performancetracker.co.uk/switch-group/28'],
        #  ['Richard Ogden', 'https://app.performancetracker.co.uk/switch-group/124'],
        #  ['Margaret Mckergan', 'https://app.performancetracker.co.uk/switch-group/134'],
        #  ['Nick Chindavata', 'https://app.performancetracker.co.uk/groups/39'],
        # ['Trouteaud Opticians', 'https://app.performancetracker.co.uk/branches/519'],
        ['Hakim Group', 'https://app.performancetracker.co.uk/groups/5'],
        # ['Shephard & Akay', 'https://app.performancetracker.co.uk/switch-branch/303']
        # ['Straine Opticians', 'https://app.performancetracker.co.uk/branches/539'],
        ]

Defining start/end date of scrape

In [ ]:
# @title Select Weekly or Daily Data Pull
time_delimeter = "Weekly" # @param ["Weekly","Daily"]


In [ ]:
start_date = datetime.date(2023, 1, 1)  # date(year, month, date)
end_date = datetime.date(2023, 12, 30)
date = start_date
dates = []
if time_delimeter == "Daily":
  datetime_delimeter = datetime.timedelta(days=1)
if time_delimeter == "Weekly":
  datetime_delimeter = datetime.timedelta(days=7)
print("dates to be scraped:")
while date < end_date+datetime_delimeter:
  dates.append(date)
  # print(date)
  date = date + datetime_delimeter
print(dates)


dates to be scraped:
[datetime.date(2023, 1, 1), datetime.date(2023, 1, 8), datetime.date(2023, 1, 15), datetime.date(2023, 1, 22), datetime.date(2023, 1, 29), datetime.date(2023, 2, 5), datetime.date(2023, 2, 12), datetime.date(2023, 2, 19), datetime.date(2023, 2, 26), datetime.date(2023, 3, 5), datetime.date(2023, 3, 12), datetime.date(2023, 3, 19), datetime.date(2023, 3, 26), datetime.date(2023, 4, 2), datetime.date(2023, 4, 9), datetime.date(2023, 4, 16), datetime.date(2023, 4, 23), datetime.date(2023, 4, 30), datetime.date(2023, 5, 7), datetime.date(2023, 5, 14), datetime.date(2023, 5, 21), datetime.date(2023, 5, 28), datetime.date(2023, 6, 4), datetime.date(2023, 6, 11), datetime.date(2023, 6, 18), datetime.date(2023, 6, 25), datetime.date(2023, 7, 2), datetime.date(2023, 7, 9), datetime.date(2023, 7, 16), datetime.date(2023, 7, 23), datetime.date(2023, 7, 30), datetime.date(2023, 8, 6), datetime.date(2023, 8, 13), datetime.date(2023, 8, 20), datetime.date(2023, 8, 27), datetime.

In [ ]:
print(dates[1].weekday())

0


In [ ]:
if time_delimeter == "Daily":
  from_dates = dates
  to_dates = dates
if time_delimeter == "Weekly":
  from_dates = []
  to_dates = []
  for date in dates:
    week_monday = date-datetime.timedelta(days=date.weekday())
    week_sunday = week_monday + datetime.timedelta(days=5)
    from_dates.append(week_monday)
    to_dates.append(week_sunday)

print("from dates: "+str(from_dates))
print("to dates: "+str(to_dates))

from dates: [datetime.date(2022, 12, 26), datetime.date(2023, 1, 2), datetime.date(2023, 1, 9), datetime.date(2023, 1, 16), datetime.date(2023, 1, 23), datetime.date(2023, 1, 30), datetime.date(2023, 2, 6), datetime.date(2023, 2, 13), datetime.date(2023, 2, 20), datetime.date(2023, 2, 27), datetime.date(2023, 3, 6), datetime.date(2023, 3, 13), datetime.date(2023, 3, 20), datetime.date(2023, 3, 27), datetime.date(2023, 4, 3), datetime.date(2023, 4, 10), datetime.date(2023, 4, 17), datetime.date(2023, 4, 24), datetime.date(2023, 5, 1), datetime.date(2023, 5, 8), datetime.date(2023, 5, 15), datetime.date(2023, 5, 22), datetime.date(2023, 5, 29), datetime.date(2023, 6, 5), datetime.date(2023, 6, 12), datetime.date(2023, 6, 19), datetime.date(2023, 6, 26), datetime.date(2023, 7, 3), datetime.date(2023, 7, 10), datetime.date(2023, 7, 17), datetime.date(2023, 7, 24), datetime.date(2023, 7, 31), datetime.date(2023, 8, 7), datetime.date(2023, 8, 14), datetime.date(2023, 8, 21), datetime.date(20

Defining which location to include sunday data for/exclude data from upload

In [ ]:
include_sunday_data = [1149001,1168002]  # locations to include sunday data for
remove_all_data = []  # locations to remove all data from upload

Defining functions to be used in scraping algorithm

In [ ]:
def login(username, password):  # go to and log into PT with login details
  dr.get("https://app.performancetracker.co.uk/login")
  email_elmt = dr.find_element(By.ID, "email")
  password_elmt = dr.find_element(By.ID, "password")
  email_elmt.send_keys(username)
  password_elmt.send_keys(password + Keys.ENTER)

In [ ]:
def generate_report():  # click generate report in PT
  generate = dr.find_element(By.XPATH, '//button[@class="btn btn-primary btn-raised btn-block"]')
  generate.click()

In [ ]:
def check_date(driver, target_date):  # cycle through date panel until month is correct
  time.sleep(0.25)
  next_month = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[1]/div[1]/button[2]')  # next month button
  prev_month = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[1]/div[1]/button[1]')  # previous month button
  time.sleep(0.25)
  current_date = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[1]/div[1]/div/div/div')  # current date selection
  print("Current month selection: "+current_date.text)
  curr_dt = parse(current_date.text)
  # print(curr_dt)
  if curr_dt.year < target_date.year:  # reqad current date and select previous or next month as appropriate
    dr.execute_script("arguments[0].click();", next_month)
    return check_date(driver, target_date)

  if curr_dt.year > target_date.year:
      pri_month = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[1]/div[1]/button[1]')
      dr.execute_script("arguments[0].click();", pri_month)
      return check_date(driver, target_date)

  if curr_dt.month < target_date.month:
      dr.execute_script("arguments[0].click();", next_month)
      return check_date(driver, target_date)

  if curr_dt.month > target_date.month:
      pri_month = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[1]/div[1]/button[1]')
      dr.execute_script("arguments[0].click();", pri_month)
      return check_date(driver, target_date)

  if curr_dt.month == target_date.month and curr_dt.year == target_date.year:  # if month is correct, continue
    print('correct')
    return
  print('weird, continuing')
  return

In [ ]:
def select_date(driver, date):
  # time.sleep(0.5)  # ensure button is ready to click
  date_n = driver.find_elements(By.XPATH, "//*[contains(text(), " +str(date)+ ")]")  # find all date buttons
  for element in date_n:  # cycle through all date buttons
    # print(element.text)
    if element.text == str(date):  # if string in button matches target date save reference to button
      date_element = element
  date_element.text
  driver.execute_script("arguments[0].click();", date_element)  # click correct date button
  return

Scrape PT data and save in PT Daily Data format as PT_Daily_Data_Scrape.xlsx

In [ ]:
# dr.close
options = webdriver.ChromeOptions()  # web driver setup options
prefs = {'download.default_directory' : '/content/'}
options.add_experimental_option('prefs', prefs)
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
dr = webdriver.Chrome(options=options)
try:
  login(USR,PW)
except Exception as exc:
  print('PT Login Failed, Error:')
  print(exc)
  print('Check Login Details are Correct')

# date = first_monday  # start with first date
with pd.ExcelWriter("PT_Daily_Data_Scrape.xlsx") as writer:  # open daily scrape file to save data
  for i in range(len(from_dates)):
  # while date < final_sunday+datetime.timedelta(days=1):  # iterate from start to final date inclusive
    # print(date)
    try:
      target_from_date = from_dates[i]
    except Exception as exc:
      print('PT Login Failed, Error:')
      print(exc)
      print('Check Login Details are Correct')
      sys.exit()
    target_to_date = to_dates[i]
    print(target_from_date)
    print(target_to_date)
    all_groups_data = pd.DataFrame()
    for group in links:  # if multiple groups selected iterate through each of them for each date
      print(group[0])  # print group name
      dr.get(group[1])  # go to group link
      dr.get('https://app.performancetracker.co.uk/reports/180')  # go to Daily KPI and Sales report
      # time.sleep(5)
      try:
        from_date = dr.find_element(By.NAME, 'from-date')  # try to find from date box, if not there it is likely that login details incorrect so scraper hasn't logged in
      except Exception as exc:
        print('PT Login Failed, Error:')
        print(exc)
        print('Check Login Details are Correct')
        quit()
        time.sleep(10)
      # from_date = dr.find_element(By.NAME, 'from-date')
      to_date = dr.find_element(By.NAME, 'to-date')  # find to date box

      print('to date')
      dr.execute_script("arguments[0].click();", to_date)  # click to date box to open date selector
      try:
        check_date(dr, target_to_date)  # run algorithm to select correct month and year, if it doesn't work at this stage it may be due to a popup
      except:
        print('exception reached')
        # dr.execute_script("arguments[0].click();", clear_notifications)
        dr.execute_script("arguments[0].click();", dr.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/header/div[2]/div/div/div/button'))  # close popup
        dr.execute_script("arguments[0].click();", dr.find_element(By.XPATH, '/html/body/div/div/div[3]/header/div[1]/div/a')) # x new reports
        dr.execute_script("arguments[0].click();", dr.find_element(By.XPATH, '/html/body/div/div/div[3]/header/div[1]/div/ul/div[1]'))
      select_date(dr,target_to_date.day)  # click correct date
      ok_button = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[2]/button[2]')  # find ok button
      dr.execute_script("arguments[0].click();", ok_button)  # click ok button



      print('from date')
      dr.execute_script("arguments[0].click();", from_date)  # open from date box
      check_date(dr, target_from_date)  # correct month
      select_date(dr,target_from_date.day)  # select date
      ok_button = dr.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div[2]/button[2]')  # find new ok button
      dr.execute_script("arguments[0].click();", ok_button)  # click ok button



      generate_report()  # click generate report
      hidden_elements = np.zeros(4)
      slept_time = 0
      while len(hidden_elements) < 5:  # once report has loaded there will be a new hidden element
        # print('waiting')
        time.sleep(5)
        slept_time = slept_time + 5
        soup = BeautifulSoup(dr.page_source)  # parse page source
        hidden_elements = soup(text=re.compile('react-empty:'))  # find hidden elements
        print('Waited '+str(slept_time)+'s, report not yet loaded')  # no. hidden elmts: '+str(len(hidden_elements)))
      print('now loaded, getting data')

      download = dr.find_element(By.XPATH, "//*[contains(text(), 'Download CSV')]")  # find download csv button
      download.click()  # click download csv

      downloaded = 0
      # iterations = 0
      while downloaded < 1:  # check whether download complete
        # iterations  = iterations+1
        # print(iterations)
        for file in os.listdir('/content/'):
          # print(iterations)
          if "download" in file:
            print(file)
            downloaded = 1  # if file "download"
        time.sleep(1)


      for file in os.listdir('/content/'):
        if "download" in file:
          print(file)
          daily_data = pd.read_csv('/content/'+file, skiprows=1)  # read downloaded file
          time.sleep(1)
          os.remove('/content/'+file)

      if len(all_groups_data) == 0:  # if there is already data from another group saved, merge data, else keep single group's file
        all_groups_data = daily_data
      else:
        all_groups_data = pd.merge(all_groups_data, daily_data, on="Metrics")

    all_groups_data.to_excel(writer, sheet_name=target_from_date.strftime('%d.%m.%Y'), index=False)  # once iterated for all groups for a date, save the data in excel file with date as sheet name
    # date = date + datetime.timedelta(days=1)  # next date

dr.close

PT to Workday Account Names Dictionary

In [ ]:
account_codes = {"Sight Tests": "Total Sight Tests",  # {Workday Account name: PT Account name}
                "ADV": "Average Dispense Value (ADV)",
                "Cost of Sales": "Supplier Invoices (ex VAT)",
                "Gross Margin": "Gross Margin",
                 "Optom DER %": "Optom DER (%)",
                 "Practice DER %": "Practice DER (%)",
                 "Turnover Inc VAT": "Turnover (inc VAT)",
                 "Turnover Ex VAT": "Turnover (ex VAT)",
                 "Available Appointments": "Available Appointments",
                 "Booked Appointments": "Total Appointments Booked",
                 "New Patients": "Number of New Patients",
                 "Refunds": "Refunds",
                 "FTA": "FTA",
                 "Optom Number of Dispenses": "How Many Advised Specs",
                 "Number of Dispenses": "How Many Dispensed Specs",
                 "Value of Dispenses": "Total Specs Sales"}

Get List of PT Practices from Daily PT Data 2024 (linked template).xlsx (ensure location codes/names in columns C/D)

In [ ]:
pt_practices = pd.read_excel('/content/Historic PT Scrape Codes.xlsx',sheet_name='Sheet 1',usecols='C,D')
pt_practice_codes = pt_practices['Unnamed: 2'].dropna()  # drop blanks/errors
pt_practice_codes

Convert Scraped data saved in PT Daily Data format to Workday upload format

In [ ]:
mapping_master = pd.read_excel('/content/Mapping Master.xlsx')  # read mapping master to get PT Name associated with each Workday Location Code

# date = first_monday
locations_not_in_PT_export = []
weekly_upload_df = pd.DataFrame()
# while date < final_sunday+datetime.timedelta(days=1):  # for each date
for i in range(len(from_dates)):
  date = from_dates[i]
  print(date)
  daily_upload_df = pd.DataFrame()  # blank daily dataframe
  day_data = pd.read_excel('/content/PT_Daily_Data_Scrape.xlsx',sheet_name=date.strftime('%d.%m.%Y'),index_col=0)  # ,skiprows=1  # read PT data for date from saved excel sheet
  data_with_codes = day_data.transpose().join(mapping_master[['PT 2023 Name','Workday Entity Code','Workday Location Code']].set_index('PT 2023 Name'))  # add location codes to pt data sheet
  for location in pt_practice_codes:  # for each location code  # switched from daily pt scraper
    for account in account_codes.keys():  # for each account
      if len(data_with_codes.loc[data_with_codes['Workday Location Code'] == location][account_codes[account]]) == 0:  # if there is no data found for this location and account
        if location not in locations_not_in_PT_export:  # and it hasn't already been added to the list of locations without data
          locations_not_in_PT_export.append(location)  # add to list
      daily_upload_df = pd.concat([daily_upload_df, pd.DataFrame({"Account": account,  # add line to dataframe with account, level code, location code, and data
                                    "Level Code": data_with_codes.loc[data_with_codes['Workday Location Code'] == location]["Workday Entity Code"],
                                    "Location Code": data_with_codes.loc[data_with_codes['Workday Location Code'] == location]["Workday Location Code"],
                                    date.strftime('%d/%m/%Y'): data_with_codes.loc[data_with_codes['Workday Location Code'] == location][account_codes[account]]})])  # data column name is the date



  if len(weekly_upload_df) == 0:  # if no data in weekly upload sheet yet then current day is full sheet
    weekly_upload_df = daily_upload_df
  else:  # merge current days data with previous days, merge on account and location code, level code doesn't need adding again
    weekly_upload_df = pd.merge(weekly_upload_df, daily_upload_df.drop(columns=["Level Code"]), how='left', left_on=["Account", "Location Code"], right_on=["Account", "Location Code"])
  date = date + datetime.timedelta(days=1)  # next date

print('Locations not in PT Export:')
print(locations_not_in_PT_export)  # print missing locations

Locations in uploaded list but not in PT Export:

In [ ]:
print(locations_not_in_PT_export)

Remove Sunday data for all practices apart from exceptions, and remove all data for practices in list

In [ ]:
# # date = first_monday
# # while date < final_sunday+datetime.timedelta(days=1):  #
# for i in range(len(from_dates)):
#   date = from_dates[i]
#   weekly_upload_df.loc[weekly_upload_df['Location Code'].isin(remove_all_data) , date.strftime('%d/%m/%Y')] = 0  # remove all data for locations in "remove_all_data" list
#   if date.weekday() == 6:
#     weekly_upload_df.loc[~weekly_upload_df['Location Code'].isin(include_sunday_data) , date.strftime('%d/%m/%Y')] = 0  # include sunday data for locations in "include_sunday_data"
#   date = date + datetime.timedelta(days=1)

Save as curr_week_daily_PT_data.xlsx

In [ ]:
weekly_upload_df.to_excel("PT_Weekly_Data_WD_Upload.xlsx", index=False)  # save week's workday data
weekly_upload_df